# Vertical mixing in numerical models

## As a mixing time scale

### Mathematical background

Imagine a two box system with unequal concentrations. Call them $c_1$ and $c_2$. To build a conceptual model, let's imagine that the gradient between the two boxes decays according to first order kinetics, that is:

$$ \frac{\partial}{\partial t}(c_1 - c_2) = -k(c_1 - c_2) $$

This assumes that the gradient follows an exponential decay in time. This makes a certain amount of intuitive sense, as we'd expect larger gradients to lead to faster changes in concentration to equalize than would smaller gradients.

To calculate the change in $c_1$ and $c_2$, we distribute the derivative:

$$ \frac{\partial c_1}{\partial t} - \frac{\partial c_2}{\partial t} = -k(c_1 - c_2) $$

Then we must recognize that in order to satisfy mass conservation, $\partial c_1/\partial t = -\partial c_2/\partial t$, which gives us:

$$
2 \frac{\partial c_1}{\partial t} = -k(c_1 - c_2) \\ 
\frac{\partial c_1}{\partial t} = -\frac{\partial c_2}{\partial t} = -\frac{k}{2}(c_1 - c_2) 
$$

Check that the signs agree with our intuition: if $c_1 > c_2$, $\partial c_1/\partial c_2$ should be $< 0$, which is the case here.

Thus, in order to calcuate the values of $c_1$ and $c_2$ at any time $t$, we need to solve these differential equations. Unlike a normal first order decay, these don't have nice simple solutions because it is the difference, not the individual concentrations, that matter. We do know how the difference between the boxes should vary though - that _does_ follow a simple exponential decay.

Let $c_1 - c_2 = \Delta c$. Then (switching to regular derivatives instead of partial because now we only have two variables):
$$
\frac{d\Delta c}{dt} = -k\Delta c \\
\Rightarrow \Delta c(t) = \Delta c(t_0) e^{-kt}
$$


### Programmatic implementation

The analytical solution for this two-box problem is easy to solve, but hard to generalize to arbitrarily many boxes. That's why numerical methods are so valuable; they are much easier to expand to an arbitrary number of boxes. Generally, numerical methods work by evaluating the derivatives at the current time, then using that derivative value to project how each quantity will change over some fixed time step. The time step must be small enough that the derivative does not change much over it's duration.

A simple implementation of the above equations (in Julia, which is kind of similar to Matlab) looks like:

In [1]:
using Plots;

dt = 0.1;  # time step = 0.1 second
run_time = 50;  # how long the model should simulate in seconds
nsteps = convert(Int64, run_time/dt);  # how many steps the model needs to take. 
                                       # Julia is a bit picky about types compared to Matlab, so convert from float 
                                       # to integer (or the indexing in the for loop fails)
c = zeros(nsteps+1, 2);  # set up an array to hold the concentration in both boxes over the modeled time
c[1,1] = 10.0;  # initial condition: box 1 has a concentration of 10.0 (arbitrary units) and box 2 has nothing
k = 0.1;  # mixing rate constant in s^{-1}
for n=1:nsteps
    dc_dt = -k*(c[n,1] - c[n,2]);  # calculate the change in concentration at timestep i
    c[n+1, 1] = c[n, 1] + dc_dt * dt;  # calculate the box 1 concentration at the next timestep, 
                                       # assuming dc_dt is constant over the timestep
    c[n+1, 2] = c[n, 2] - dc_dt * dt;  # calculate the box 2 concentration at the next timestep, 
                                       # remembering dc_2/dt = - dc_1/dt
end

time_vector = (0.0:nsteps)*dt;
plot(time_vector, c[:,1], label="Box 1", xlabel="Time (s)", ylabel="Concentration (arbitrary units)")
plot!(time_vector, c[:,2], label="Box 2")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 Time (s) 
 
 
 Concentration (arbitrary units) 
 
<polyline clip-path="url(#clip1502)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 44.8772,11.811 45.9481,15.2941 47.019,18.7076 48.0899,22.0527 49.1607,25.331 50.2316,28.5437 51.3025,31.6921 52.3734,34.7776 53.4442,37.8014 54.5151,40.7647 
 55.586,43.6687 56.6569,46.5146 57.7277,49.3036 58.7986,52.0369 59.8695,54.7155 60.9404,57.3405 62.0112,59.913 63.0821,62.434 64.153,64.9047 65.2239,67.3259 
 66.2947,69.6987 67.3656,72.024 68.4365,74.3029 69.5074,76.5361 70.5782,78.7247 71.6491,80.8695 72.72,82.9715 73.7909,85.0314 74.8617,87.0501 75.9326,89.0284 
 77.0035,90.9671 78.0744,92.8671 79.1452,94.7291 80.2161,96.5538 81.287,98.342 82.3579,100.095 83.4287,101.812 84.4996,103.495 85.5705,105.144 86.6414,106.761 
 87.7123,108.345 88.7831,109.897 89.854,111.419 90.9249,112.91 91.9958,114.371 93.0666,115.803 94.1375,117.206 95.2084,118.581 96.2793,119.929 97.3501,121.25 
 98.421,122.544 99.4919,123.812 100.563,125.055 101.634,126.274 102.705,127.467 103.775,128.637 104.846,129.784 105.917,130.908 106.988,132.009 108.059,133.088 
 109.13,134.146 110.201,135.182 111.272,136.198 112.342,137.193 113.413,138.168 114.484,139.124 115.555,140.061 116.626,140.979 117.697,141.879 118.768,142.761 
 119.839,143.625 120.909,144.472 121.98,145.302 123.051,146.115 124.122,146.912 125.193,147.693 126.264,148.458 127.335,149.209 128.406,149.944 129.476,150.664 
 130.547,151.37 131.618,152.062 132.689,152.74 133.76,153.405 134.831,154.056 135.902,154.694 136.973,155.32 138.043,155.932 139.114,156.533 140.185,157.122 
 141.256,157.699 142.327,158.264 143.398,158.818 144.469,159.361 145.54,159.893 146.61,160.415 147.681,160.926 148.752,161.426 149.823,161.917 150.894,162.398 
 151.965,162.87 153.036,163.331 154.107,163.784 155.177,164.228 156.248,164.663 157.319,165.089 158.39,165.506 159.461,165.915 160.532,166.316 161.603,166.709 
 162.674,167.094 163.744,167.472 164.815,167.842 165.886,168.204 166.957,168.559 168.028,168.908 169.099,169.249 170.17,169.583 171.241,169.911 172.311,170.232 
 173.382,170.547 174.453,170.855 175.524,171.157 176.595,171.453 177.666,171.744 178.737,172.028 179.808,172.307 180.878,172.58 181.949,172.848 183.02,173.11 
 184.091,173.367 185.162,173.619 186.233,173.866 187.304,174.108 188.375,174.345 189.445,174.578 190.516,174.805 191.587,175.029 192.658,175.247 193.729,175.462 
 194.8,175.672 195.871,175.878 196.942,176.079 198.012,176.277 199.083,176.471 200.154,176.661 201.225,176.847 202.296,177.029 203.367,177.208 204.438,177.383 
 205.509,177.555 206.579,177.723 207.65,177.888 208.721,178.049 209.792,178.208 210.863,178.363 211.934,178.515 213.005,178.664 214.076,178.81 215.146,178.953 
 216.217,179.093 217.288,179.231 218.359,179.366 219.43,179.498 220.501,179.627 221.572,179.754 222.643,179.878 223.713,180 224.784,180.119 225.855,180.236 
 226.926,180.351 227.997,180.463 229.068,180.573 230.139,180.681 231.21,180.787 232.28,180.89 233.351,180.992 234.422,181.091 235.493,181.189 236.564,181.284 
 237.635,181.378 238.706,181.47 239.777,181.559 240.847,181.648 241.918,181.734 242.989,181.819 244.06,181.902 245.131,181.983 246.202,182.062 247.273,182.141 
 248.344,182.217 249.414,182.292 250.485,182.366 251.556,182.438 252.627,182.508 253.698,182.577 254.769,182.645 255.84,182.711 256.911,182.777 257.981,182.84 
 259.052,182.903 260.123,182.964 261.194,183.024 262.265,183.083 263.336,183.141 264.407,183.197 265.478,183.252 266.548,183.307 267.619,183.36 268.69,183.412 
 269.761,183.463 270.832,183.513 271.903,183.562 272.974,183.61 274.045,183.657 275.115,183.704 276.186,183.749 277.257,183.793 278.328,183.837 279.399,183.879 
 280.47,183.921 281.541,183.962 282.612,184.002 283.682,184.041 284.753,184.08 285.824,18

As we'd expect, the concentrations in the two boxes converge. How does this compare to the analytical solution? Well, we only have an analytical solution for the difference between the boxes, so that's what we'll compare:

In [2]:
delta_c = c[:,1] - c[:,2]
analytical_soln = delta_c[1] * e.^(-k * time_vector)
plot(time_vector, delta_c, label="Numerical solution", xlabel="Time (s)", ylabel="Delta C")
plot!(time_vector, analytical_soln, label="Analytical solution")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 Time (s) 
 
 
 Delta C 
 
<polyline clip-path="url(#clip1702)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 44.8772,11.811 45.9481,18.7775 47.019,25.6047 48.0899,32.2953 49.1607,38.8521 50.2316,45.2777 51.3025,51.5749 52.3734,57.7461 53.4442,63.7938 54.5151,69.7207 
 55.586,75.5289 56.6569,81.2211 57.7277,86.7993 58.7986,92.266 59.8695,97.6234 60.9404,102.874 62.0112,108.019 63.0821,113.061 64.153,118.003 65.2239,122.845 
 66.2947,127.591 67.3656,132.242 68.4365,136.8 69.5074,141.267 70.5782,145.644 71.6491,149.934 72.72,154.138 73.7909,158.258 74.8617,162.295 75.9326,166.252 
 77.0035,170.13 78.0744,173.93 79.1452,177.654 80.2161,181.304 81.287,184.88 82.3579,188.385 83.4287,191.82 84.4996,195.187 85.5705,198.486 86.6414,201.719 
 87.7123,204.887 88.7831,207.992 89.854,211.035 90.9249,214.017 91.9958,216.939 93.0666,219.803 94.1375,222.61 95.2084,225.36 96.2793,228.056 97.3501,230.697 
 98.421,233.286 99.4919,235.823 100.563,238.309 101.634,240.746 102.705,243.133 103.775,245.473 104.846,247.767 105.917,250.014 106.988,252.216 108.059,254.375 
 109.13,256.49 110.201,258.563 111.272,260.594 112.342,262.585 113.413,264.536 114.484,266.448 115.555,268.322 116.626,270.158 117.697,271.958 118.768,273.721 
 119.839,275.45 120.909,277.143 121.98,278.803 123.051,280.43 124.122,282.024 125.193,283.586 126.264,285.117 127.335,286.617 128.406,288.088 129.476,289.529 
 130.547,290.941 131.618,292.325 132.689,293.681 133.76,295.01 134.831,296.312 135.902,297.589 136.973,298.84 138.043,300.066 139.114,301.267 140.185,302.444 
 141.256,303.598 142.327,304.729 143.398,305.837 144.469,306.923 145.54,307.987 146.61,309.03 147.681,310.052 148.752,311.054 149.823,312.036 150.894,312.998 
 151.965,313.94 153.036,314.864 154.107,315.77 155.177,316.657 156.248,317.527 157.319,318.379 158.39,319.214 159.461,320.032 160.532,320.834 161.603,321.62 
 162.674,322.391 163.744,323.146 164.815,323.885 165.886,324.61 166.957,325.321 168.028,326.017 169.099,326.699 170.17,327.368 171.241,328.023 172.311,328.666 
 173.382,329.295 174.453,329.912 175.524,330.516 176.595,331.109 177.666,331.689 178.737,332.258 179.808,332.816 180.878,333.362 181.949,333.898 183.02,334.422 
 184.091,334.937 185.162,335.44 186.233,335.934 187.304,336.418 188.375,336.893 189.445,337.358 190.516,337.813 191.587,338.26 192.658,338.697 193.729,339.126 
 194.8,339.546 195.871,339.958 196.942,340.361 198.012,340.757 199.083,341.144 200.154,341.524 201.225,341.896 202.296,342.261 203.367,342.619 204.438,342.969 
 205.509,343.312 206.579,343.649 207.65,343.978 208.721,344.302 209.792,344.618 210.863,344.929 211.934,345.233 213.005,345.531 214.076,345.823 215.146,346.109 
 216.217,346.39 217.288,346.665 218.359,346.934 219.43,347.198 220.501,347.457 221.572,347.71 222.643,347.959 223.713,348.202 224.784,348.441 225.855,348.675 
 226.926,348.904 227.997,349.129 229.068,349.349 230.139,349.564 231.21,349.776 232.28,349.983 233.351,350.186 234.422,350.385 235.493,350.58 236.564,350.771 
 237.635,350.958 238.706,351.142 239.777,351.322 240.847,351.498 241.918,351.671 242.989,351.84 244.06,352.006 245.131,352.169 246.202,352.328 247.273,352.484 
 248.344,352.637 249.414,352.787 250.485,352.934 251.556,353.078 252.627,353.219 253.698,353.357 254.769,353.493 255.84,353.626 256.911,353.756 257.981,353.884 
 259.052,354.009 260.123,354.131 261.194,354.251 262.265,354.369 263.336,354.484 264.407,354.597 265.478,354.708 266.548,354.817 267.619,354.923 268.69,355.027 
 269.761,355.129 270.832,355.229 271.903,355.328 272.974,355.424 274.045,355.518 275.115,355.61 276.186,355.701 277.257,355.789 278.328,355.876 279.399,355.962 
 280.47,356.045 281.541,356.127 282.612,356.207 283.682,356.285 284.753,356.362 285.824,356.438 286.895,356.512 287.966,356.584 

The numerical solution converges faster than the analytical one. This makes a certain sense - because the numerical solution assumes that the $d\Delta c/dt$ stays constant over each time step, and each $d\Delta c/dt$ is computed at the beginning of it's timestep, the change is overestimated.

### More boxes and the diffusion equation

Having only two boxes is boring - what about adding more? For now, we'll stick to a 1D model, meaning that each box has two neighbors - top and bottom, except the first and last boxes obviously. For now, let's focus on the boxes that do have two neighbors. Amending the solution to allow for multiple boxes is easy, we just need to sum the contributions from both neighbors. We'll use a more general index notation now, using a subscript $i$ to indicate the $i$-th box:

$$ \frac{\partial c_i}{\partial t} = -k(c_i - c_{i-1}) - k(c_i - c_{i+1}) $$

In [3]:
dt = 0.1;  # time step = 0.1 second
run_time = 500;  # how long the model should simulate in seconds
nsteps = convert(Int64, run_time/dt);  # how many steps the model needs to take. 
                                       # Julia is a bit picky about types compared to Matlab, so convert from float 
                                       # to integer (or the indexing in the for loop fails)
nboxes = 10;  # how many boxes to include in the model
c = zeros(nsteps+1, nboxes);  # set up an array to hold the concentration in both boxes over the modeled time
c[1,1] = 10.0;  # initial condition: box 1 has a concentration of 10.0 (arbitrary units) and box 2 has nothing
k = 0.1;  # mixing rate constant in s^{-1}
for n=1:nsteps
    for i=1:nboxes
        dc_dt = 0.0;
        if i > 1
            dc_dt += -k*(c[n,i] - c[n,i-1]); # calculate the change in concentration at timestep n due to the box 
                                             # below as long as we're not in the bottom box
        end
        if i < nboxes
            dc_dt += -k*(c[n,i] - c[n,i+1]); # calculate the change in concentration at timestep n due to the box 
                                             # above as long as we're not in the top box
        end
        c[n+1, i] = c[n, i] + dc_dt * dt;  # calculate the box i concentration at the next timestep, 
                                           # assuming dc_dt is constant over the timestep
    end
end

time_vector = (0.0:nsteps)*dt;
plot(time_vector, c, xlabel="Time (s)", ylabel="Concentration (arbitrary units)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 500 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 Time (s) 
 
 
 Concentration (arbitrary units) 
 
<polyline clip-path="url(#clip1902)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 44.8772,11.811 44.9843,15.2941 45.0914,18.7076 45.1985,22.0531 45.3056,25.3324 45.4127,28.5471 45.5198,31.6988 45.6268,34.7891 45.7339,37.8195 45.841,40.7914 
 45.9481,43.7063 46.0552,46.5656 46.1623,49.3706 46.2694,52.1227 46.3765,54.823 46.4835,57.473 46.5906,60.0737 46.6977,62.6263 46.8048,65.1321 46.9119,67.5921 
 47.019,70.0073 47.1261,72.3789 47.2332,74.7079 47.3402,76.9952 47.4473,79.2419 47.5544,81.4488 47.6615,83.617 47.7686,85.7472 47.8757,87.8405 47.9828,89.8975 
 48.0899,91.9192 48.1969,93.9064 48.304,95.8598 48.4111,97.7802 48.5182,99.6684 48.6253,101.525 48.7324,103.351 48.8395,105.146 48.9466,106.913 49.0536,108.65 
 49.1607,110.359 49.2678,112.041 49.3749,113.695 49.482,115.324 49.5891,116.926 49.6962,118.503 49.8033,120.056 49.9103,121.584 50.0174,123.089 50.1245,124.57 
 50.2316,126.029 50.3387,127.466 50.4458,128.881 50.5529,130.275 50.66,131.647 50.767,133 50.8741,134.332 50.9812,135.645 51.0883,136.939 51.1954,138.214 
 51.3025,139.471 51.4096,140.71 51.5167,141.931 51.6237,143.134 51.7308,144.321 51.8379,145.491 51.945,146.645 52.0521,147.783 52.1592,148.906 52.2663,150.013 
 52.3734,151.105 52.4804,152.182 52.5875,153.245 52.6946,154.293 52.8017,155.328 52.9088,156.349 53.0159,157.357 53.123,158.351 53.2301,159.333 53.3371,160.302 
 53.4442,161.259 53.5513,162.203 53.6584,163.136 53.7655,164.057 53.8726,164.966 53.9797,165.864 54.0868,166.751 54.1938,167.627 54.3009,168.493 54.408,169.347 
 54.5151,170.192 54.6222,171.027 54.7293,171.851 54.8364,172.666 54.9435,173.471 55.0505,174.267 55.1576,175.053 55.2647,175.831 55.3718,176.599 55.4789,177.359 
 55.586,178.11 55.6931,178.852 55.8002,179.587 55.9073,180.313 56.0143,181.03 56.1214,181.74 56.2285,182.443 56.3356,183.137 56.4427,183.824 56.5498,184.504 
 56.6569,185.176 56.764,185.841 56.871,186.499 56.9781,187.15 57.0852,187.795 57.1923,188.432 57.2994,189.063 57.4065,189.687 57.5136,190.305 57.6207,190.917 
 57.7277,191.523 57.8348,192.122 57.9419,192.715 58.049,193.303 58.1561,193.884 58.2632,194.46 58.3703,195.031 58.4774,195.595 58.5844,196.154 58.6915,196.708 
 58.7986,197.257 58.9057,197.8 59.0128,198.338 59.1199,198.871 59.227,199.399 59.3341,199.922 59.4411,200.44 59.5482,200.953 59.6553,201.462 59.7624,201.966 
 59.8695,202.465 59.9766,202.96 60.0837,203.451 60.1908,203.937 60.2978,204.418 60.4049,204.896 60.512,205.369 60.6191,205.838 60.7262,206.303 60.8333,206.764 
 60.9404,207.221 61.0475,207.674 61.1545,208.123 61.2616,208.568 61.3687,209.01 61.4758,209.448 61.5829,209.882 61.69,210.312 61.7971,210.739 61.9042,211.162 
 62.0112,211.582 62.1183,211.999 62.2254,212.412 62.3325,212.822 62.4396,213.228 62.5467,213.631 62.6538,214.031 62.7609,214.428 62.8679,214.822 62.975,215.212 
 63.0821,215.6 63.1892,215.984 63.2963,216.366 63.4034,216.744 63.5105,217.12 63.6176,217.493 63.7246,217.863 63.8317,218.23 63.9388,218.594 64.0459,218.956 
 64.153,219.315 64.2601,219.671 64.3672,220.024 64.4743,220.375 64.5813,220.724 64.6884,221.07 64.7955,221.413 64.9026,221.754 65.0097,222.092 65.1168,222.428 
 65.2239,222.762 65.331,223.093 65.438,223.422 65.5451,223.749 65.6522,224.073 65.7593,224.395 65.8664,224.715 65.9735,225.032 66.0806,225.348 66.1877,225.661 
 66.2947,225.972 66.4018,226.281 66.5089,226.588 66.616,226.893 66.7231,227.195 66.8302,227.496 66.9373,227.795 67.0444,228.091 67.1514,228.386 67.2585,228.679 
 67.3656,228.97 67.4727,229.259 67.5798,229.546 67.6869,229.831 67.794,230.115 67.9011,230.396 68.0081,230.676 68.1152,230.954 68.2223,231.23 68.3294,231.505 
 68.4365,231.778 68.5436,232.049 68.6507,232.318 68.7578,232.586 68.8648,232.852 68.971

As you can see, now we get more interesting behavior! The second box is particularly cool, as initially the gradient vs. the first box is large enough that it's gaining concentration faster than it's losing it. As time goes on though, all the boxes equilibrate around 1/10th of the original concentration, as you'd expect.

Now, let's take another look at the equation:

$$ \frac{\partial c_i}{\partial t} = -k(c_i - c_{i-1}) - k(c_i - c_{i+1}) $$

This is a difference of differences, which suggests that perhaps this mixing is represented by a second derivative? In fact it is, and the diffusion equation is:

$$ \frac{\partial c}{\partial t} = D \frac{\partial^2 c}{\partial x^2} $$

where $D$ is the diffusion constant. It has units of m$^2$ s$^{-1}$. (We've dropped the negative sign because the correct sign will come out of the second derivative.)  The question is, how do we represent that $\partial^2 c/\partial x^2$ on a discrete grid, i.e. our boxes?

## The Taylor series: a formalism for discrete derivatives
### An exact solution

The Taylor series is (presenting it in terms of time since we've been thinking about time derivatives so far):

$$ f(t + \Delta t) = f(t) + \sum_{i=1}^{\infty} \frac{(\Delta t)^n}{n!} \left.\frac{d^n f(t)}{dt^n}\right|_t $$

In plain English, it says that if you know the value of the function $f$ at the point $x$, then to calculate the value of $f$ some time $\Delta t$ away from $t$, you need to add $\Delta t$ times all the derivatives of $f$, evaluated at $t$.

The infinite sum makes this seem fairly unuseful (after all, how does one calculate _infinite_ derivatives?), but there are two ways it becomes practical:

1. We know that all the derivatives after a certain order are 0
2. We're willing to arbitrarily truncate the series at some point

For an example of the first case, consider a good old mechanics problem where you have some body with a known acceleration ($a$), initial velocity ($v_0$), and initial position ($x_0$). To translate into a Taylor series, you have:

$$ x_0 = f(t) \\
v_0 = \left.\frac{df(t)}{dt}\right|_{t_0} \\
a = \left.\frac{d^2 f(t)}{dt^2}\right|_{t_0}$$

If we know acceleration is constant, then all the higher derivatives are 0, so the Taylor series becomes:

$$ f(t + \Delta t) = f(t) + \Delta t \left.\frac{df}{dt}\right|_{t_0} + \frac{(\Delta t)^2}{2} \left.\frac{d^2f}{dt^2}\right|_{t_0} \\
= x_0 + \Delta t \cdot v_0 + \frac{(\Delta t)^2}{2} a $$

which, if we take $t_0 = 0$, looks exactly like the typical mechanics equation:

$$ x(t) = x_0 + v_0t + \frac{1}{2}at^2 $$

So in that case, the Taylor series can provide an exact answer. 


### An approximate solution applied to the diffusion equation

However, in our vertical mixing case, we _don't_ know that the higher derivatives are 0, but we _could_ argue that as long as $\Delta t$ is small, the higher derivatives aren't too important. In that case, we can lop off everything after the first derivative:

$$ f(t_0 + \Delta t) = f(t_0) + \Delta t \left.\frac{df(t)}{dt}\right|_{t_0} $$

If you look back at the first code example, this is exactly how we handled this equation numerically - at each time step, we calculated $df/dt$, and added that times the time step size ($\Delta t$) to the value at the current time to get the value at the next time. Great! That means that we now have a mathematical formalism to describe how to do our numerical integration.

We can also turn this around, in a sense, and use the Taylor series to figure out how to calculate derivatives from discrete data. So we want to use to to figure out how to calculate the $d^2 c/dx^2$ value in the diffusion equation. Basically, we're going to treat the derivative like a variable and solve for it.

For example, if we wanted to get an expression for $df/dx$ from a Taylor series, what would we do? First write out a Taylor series that stops as soon as we have the derivative we want:

$$ f(x_0 + \Delta x) = f(x_0) + \Delta x \frac{df}{dx} $$

(I've switched to functions of $x$ because in the diffusion equation we're thinking about derivatives in space, not time.) Then just rearrange until $df/dx$ is isolated:

$$ \frac{f(x_0 + \Delta x) - f(x_0)}{\Delta x} =  \frac{df}{dx} $$

Presto, we have an expression for $df/dx$! Unsurprisingly, this looks a lot like a good old rise-over-run calculation, as it should.

Since we're thinking about using this in a model based on a grid, I'm going to introduce a different sort of notation. Remember that when we started talking about mixing, we thought of it as being two boxes with different concentrations of whatever inside. When we modeled it, we assumed that each box had a uniform concentration inside it, and the only difference in concentrations was between the boxes.  Later, when we added more boxes, we gave each one an index. We could rewrite the last equation using indexes instead:

$$ \frac{f_{i+1} - f_i}{\Delta x} = \frac{df}{dx} $$

Here, $f_{i+1}$ is shorthand for $f(x + \Delta x)$. The assumption is that each box has a width of $\Delta x$ so that if $f$ is only defined at each box center, then the centers are $\Delta x$ apart, or $x_{i+1} - x_i = \Delta x$.  This is just a more convenient way of writing things when our function is defined at discrete points, and so can be identified by integer indices, rather than a continuous coordinate.

Now, back to the diffusion equation. Since we want a second derivative, we need a Taylor series that goes out at least to the second derivative. In index notation:

$$ f_{i+1} = f_i + \Delta x \frac{df}{dx} + \frac{(\Delta x)^2}{2} \frac{d^2f}{dx^2} $$

We'd like $d^2f/dx^2$ to be defined entirely by values of $f$ and $\Delta x$ - in our simulation, those are easy to look up. $df/dx$ is not. So can we just replace $df/dx$ with the expression we just got? Well...

$$ f_{i+1} = f_i + \Delta x \left(\frac{f_{i+1} - f_i}{\Delta x}\right) + \frac{(\Delta x)^2}{2} \frac{d^2f}{dx^2} \\
\Rightarrow f_{i+1} - f_{i+1} = f_i + - f_i + \frac{(\Delta x)^2}{2} \frac{d^2f}{dx^2} \\
\Rightarrow 0 = \frac{d^2f}{dx^2} $$

Oops. That didn't work, because our expression for $df/dx$ wasn't independent of the expression we were solving for $d^2f/dx^2$. Another way to think about it is that to define a second derivative, we need 3 points, and right now we only have 2.

We can add in a 3rd point (or get an independent expression for $df/dx$) if we consider the Taylor series for $f_{i-1}$ instead. There's one catch: we still want to include the second derivative. Basically, whenever we start combining Taylor series, they should all be truncated at the same order:

$$ f_{i-1} = f_i + (-\Delta x)\frac{df}{dx} + \frac{(-\Delta x)^2}{2}\frac{d^2f}{dx^2} \\
\Rightarrow \frac{f_i - f_{i-1}}{\Delta x} + \frac{\Delta x}{2}\frac{d^2f}{dx^2} = \frac{df}{dx} $$

(We have $-\Delta x$ because $f_{i-1}$ is "behind" $f_i$.)

So plug this into the second derivative equation:

$$ f_{i+1} = f_i + \Delta x \frac{df}{dx} + \frac{(\Delta x)^2}{2} \frac{d^2f}{dx^2} \\
\Rightarrow f_{i+1} = f_i + \Delta x \left(\frac{f_i - f_{i-1}}{\Delta x} + \frac{\Delta x}{2}\frac{d^2f}{dx^2}\right) + \frac{(\Delta x)^2}{2} \frac{d^2f}{dx^2} \\
\Rightarrow f_{i+1} = 2f_i - f_{i-1} + 2\frac{(\Delta x)^2}{2} \frac{d^2f}{dx^2} \\
\Rightarrow \frac{f_{i+1} - 2f_i + f_{i-1}}{(\Delta x)^2} = \frac{d^2f}{dx^2}
$$

Whew! Finally, we have something that we can plug into our model right? Yes! Well, almost. Before we do that, we need to discuss...

## Boundary conditions

Right now, our expression for $d^2f/dx^2$ will fail if $i=1$, because there is no box 0, i.e. $i-1$. It'll also fail at the other end, where $i = n_x$ ($n_x$ is the number of boxes). Before, we handled this by basically saying "if you're at one of the edge boxes, only count the mass transfer from one side". Now we want to be a little bit more formal.

There's three general types of boundary conditions:

1. Dirichlet BCs: the value of $f$ is fixed at the boundary
2. Von Neumann BCs: the value of $df/dx$ is fixed at the boundary
3. Periodic BCs: $f_1$ and $f_{n_x}$ are considered neighbors, i.e. the model "wraps around" (like a globe)

### Dirichlet BCs

Dirichlet BCs are the easist to understand. In the context of vertical mixing (and mass transfer more generally) these can be thought of as an extra box adjacent to your model's edge that is so big that no amount of mass transferred to it from your model would perturb its concentration. You could, for instance, treat the stratosphere as this sort of boundary - assume that it is well mixed on the time scale of stratospheric-tropospheric exchange and that it's concentration is fixed, or even that it varies in time, but is not affected by the troposphere. (This is a gross simplification, but let's go with it for now.)

The easiest way to implement these is to pad your vector of model boxes with one extra box on each boundary and set those extra boxes to whatever you want the boundary condition to be.

In [12]:
dt = 0.1;  # time step = 0.1 second
dx = 1000;  # box width in meters
D = 0.1*dx^2;  # diffusion constant in m^2/s. Calculate the D equivalent to our old k for given dx 

run_time = 500;  # how long the model should simulate in seconds
nsteps = convert(Int64, run_time/dt);  # how many steps the model needs to take. 
                                       # Julia is a bit picky about types compared to Matlab, so convert from float 
                                       # to integer (or the indexing in the for loop fails)

nboxes = 10;  # how many boxes to include in the model
c = zeros(nsteps+1, nboxes);  # set up an array to hold the concentration in all boxes over the modeled time.
c[1,1] = 10.0;  # initial condition: box 1 has a concentration of 10.0 (arbitrary units) and box 2 has nothing
# Boundary conditions
bottom_bc = 0.0;
top_bc = 1.0;

for n=1:nsteps
    for i=1:nboxes
        c_padded = vcat(bottom_bc, c[n,:], top_bc)
        
        # Adjust the box index to be corrent in the padded vector
        ip = i + 1; 
        # Here's the implementation of the second derivative
        dc_dt = D * (c_padded[ip+1] - 2*c_padded[ip] + c_padded[ip-1])/(dx^2)
        
        # calculate the box i concentration at the next timestep, 
        # assuming dc_dt is constant over the timestep
        c[n+1, i] = c[n, i] + dc_dt * dt;  
    end
end

time_vector = (0.0:nsteps)*dt;
plot(time_vector, c, xlabel="Time (s)", ylabel="Concentration (arbitrary units)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 500 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 Time (s) 
 
 
 Concentration (arbitrary units) 
 
<polyline clip-path="url(#clip2502)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 44.8772,11.811 44.9843,18.7772 45.0914,25.5693 45.1985,32.192 45.3056,38.6502 45.4127,44.9485 45.5198,51.0912 45.6268,57.0828 45.7339,62.9273 45.841,68.6289 
 45.9481,74.1915 46.0552,79.619 46.1623,84.915 46.2694,90.0832 46.3765,95.127 46.4835,100.05 46.5906,104.855 46.6977,109.546 46.8048,114.125 46.9119,118.596 
 47.019,122.962 47.1261,127.225 47.2332,131.388 47.3402,135.454 47.4473,139.425 47.5544,143.304 47.6615,147.094 47.7686,150.797 47.8757,154.414 47.9828,157.949 
 48.0899,161.404 48.1969,164.78 48.304,168.08 48.4111,171.306 48.5182,174.459 48.6253,177.541 48.7324,180.555 48.8395,183.503 48.9466,186.385 49.0536,189.203 
 49.1607,191.96 49.2678,194.656 49.3749,197.294 49.482,199.874 49.5891,202.399 49.6962,204.869 49.8033,207.287 49.9103,209.653 50.0174,211.968 50.1245,214.234 
 50.2316,216.453 50.3387,218.625 50.4458,220.751 50.5529,222.833 50.66,224.872 50.767,226.868 50.8741,228.824 50.9812,230.739 51.0883,232.615 51.1954,234.453 
 51.3025,236.254 51.4096,238.018 51.5167,239.747 51.6237,241.441 51.7308,243.101 51.8379,244.728 51.945,246.324 52.0521,247.887 52.1592,249.42 52.2663,250.923 
 52.3734,252.397 52.4804,253.842 52.5875,255.26 52.6946,256.65 52.8017,258.014 52.9088,259.351 53.0159,260.663 53.123,261.951 53.2301,263.214 53.3371,264.453 
 53.4442,265.67 53.5513,266.864 53.6584,268.035 53.7655,269.186 53.8726,270.315 53.9797,271.423 54.0868,272.512 54.1938,273.58 54.3009,274.63 54.408,275.66 
 54.5151,276.672 54.6222,277.667 54.7293,278.643 54.8364,279.603 54.9435,280.545 55.0505,281.471 55.1576,282.381 55.2647,283.275 55.3718,284.154 55.4789,285.018 
 55.586,285.867 55.6931,286.701 55.8002,287.522 55.9073,288.328 56.0143,289.121 56.1214,289.901 56.2285,290.668 56.3356,291.422 56.4427,292.164 56.5498,292.893 
 56.6569,293.611 56.764,294.317 56.871,295.011 56.9781,295.694 57.0852,296.367 57.1923,297.028 57.2994,297.679 57.4065,298.32 57.5136,298.95 57.6207,299.571 
 57.7277,300.182 57.8348,300.784 57.9419,301.376 58.049,301.959 58.1561,302.533 58.2632,303.098 58.3703,303.655 58.4774,304.203 58.5844,304.743 58.6915,305.275 
 58.7986,305.799 58.9057,306.315 59.0128,306.824 59.1199,307.325 59.227,307.818 59.3341,308.305 59.4411,308.784 59.5482,309.256 59.6553,309.722 59.7624,310.181 
 59.8695,310.633 59.9766,311.079 60.0837,311.519 60.1908,311.952 60.2978,312.379 60.4049,312.801 60.512,313.216 60.6191,313.626 60.7262,314.03 60.8333,314.428 
 60.9404,314.821 61.0475,315.209 61.1545,315.591 61.2616,315.969 61.3687,316.341 61.4758,316.708 61.5829,317.07 61.69,317.428 61.7971,317.78 61.9042,318.129 
 62.0112,318.472 62.1183,318.811 62.2254,319.146 62.3325,319.476 62.4396,319.802 62.5467,320.124 62.6538,320.442 62.7609,320.756 62.8679,321.066 62.975,321.371 
 63.0821,321.673 63.1892,321.972 63.2963,322.266 63.4034,322.557 63.5105,322.844 63.6176,323.128 63.7246,323.409 63.8317,323.685 63.9388,323.959 64.0459,324.229 
 64.153,324.496 64.2601,324.76 64.3672,325.02 64.4743,325.278 64.5813,325.532 64.6884,325.784 64.7955,326.032 64.9026,326.278 65.0097,326.521 65.1168,326.761 
 65.2239,326.998 65.331,327.232 65.438,327.464 65.5451,327.693 65.6522,327.92 65.7593,328.144 65.8664,328.365 65.9735,328.584 66.0806,328.8 66.1877,329.014 
 66.2947,329.226 66.4018,329.435 66.5089,329.642 66.616,329.847 66.7231,330.05 66.8302,330.25 66.9373,330.448 67.0444,330.644 67.1514,330.838 67.2585,331.03 
 67.3656,331.22 67.4727,331.408 67.5798,331.593 67.6869,331.777 67.794,331.959 67.9011,332.139 68.0081,332.317 68.1152,332.493 68.2223,332.668 68.3294,332.84 
 68.4365,333.011 68.5436,333.18 68.6507,333.348 68.7578,333.513 68.8648,333.677 68.9719,33

Choosing a top boundary condition of 1.0 and a bottom one of 0.0 gives interesting results - eventually, the concentration gradient flips so that box 10 has a concentration near 1 and box 1 has one near 0, despite the initial conditions being reversed. As you'd expect, the BCs eventually drive the overall gradient, since they don't change.

### Von Neumann BCs

Von Neumann BCs allow you to specify the value of $df/dx$ at the boundary. This is similar to what we did originally, which was essentially say $df/dx = 0$ at the boundaries. These are a little more complicated to implement because we don't have anywhere in the code - yet - where we can just specify the derivative. We will do this with "ghost points" - these are again extra boxes padding the concentration vector, but instead they have their concentration calculated to be what will give the desired $df/dx$.

In [14]:
function calc_ghost_concentration(dfdx, f_edge, dx)
    # dfdx is the desired derivative
    # f_edge is the concentration at the grid point on the edge of the domain
    # dx is the grid spacing
    #
    # Assume df_dx = (f_edge - f_ghost)/dx then f_ghost = f_edge - df_dx * dx
    return f_edge - dfdx * dx;
end

function pad_von_neumann(c_vec, dfdx_bottom, dfdx_top, dx)
    # c_vec is the current vector of concentrations
    # dfdx_bottom is the bottom Von Neumann boundary condition
    # dfdx_top is the top Von Neumann boundary condition
    bc_bottom = calc_ghost_concentration(dfdx_bottom, c_vec[1], dx);
    bc_top = calc_ghost_concentration(dfdx_top, c_vec[end], dx);
    return vcat(bc_bottom, c_vec, bc_top)
end

dt = 0.1;  # time step = 0.1 second
dx = 1000;  # box width in meters
D = 0.1*dx^2;  # diffusion constant in m^2/s. Calculate the D equivalent to our old k for given dx 

run_time = 500;  # how long the model should simulate in seconds
nsteps = convert(Int64, run_time/dt);  # how many steps the model needs to take. 
                                       # Julia is a bit picky about types compared to Matlab, so convert from float 
                                       # to integer (or the indexing in the for loop fails)

nboxes = 10;  # how many boxes to include in the model
c = zeros(nsteps+1, nboxes);  # set up an array to hold the concentration in all boxes over the modeled time.
c[1,1] = 10.0;  # initial condition: box 1 has a concentration of 10.0 (arbitrary units) and box 2 has nothing
# Boundary conditions - set to no flux
bottom_bc = 0.0;
top_bc = 0.0;

for n=1:nsteps
    for i=1:nboxes
        c_padded = pad_von_neumann(c[n,:], bottom_bc, top_bc, dx);
        
        # Adjust the box index to be corrent in the padded vector
        ip = i + 1; 
        # Here's the implementation of the second derivative
        dc_dt = D * (c_padded[ip+1] - 2*c_padded[ip] + c_padded[ip-1])/(dx^2)
        
        # calculate the box i concentration at the next timestep, 
        # assuming dc_dt is constant over the timestep
        c[n+1, i] = c[n, i] + dc_dt * dt;  
    end
end

time_vector = (0.0:nsteps)*dt;
plot(time_vector, c, xlabel="Time (s)", ylabel="Concentration (arbitrary units)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 500 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 Time (s) 
 
 
 Concentration (arbitrary units) 
 
<polyline clip-path="url(#clip2702)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 44.8772,11.811 44.9843,15.2941 45.0914,18.7076 45.1985,22.0531 45.3056,25.3324 45.4127,28.5471 45.5198,31.6988 45.6268,34.7891 45.7339,37.8195 45.841,40.7914 
 45.9481,43.7063 46.0552,46.5656 46.1623,49.3706 46.2694,52.1227 46.3765,54.823 46.4835,57.473 46.5906,60.0737 46.6977,62.6263 46.8048,65.1321 46.9119,67.5921 
 47.019,70.0073 47.1261,72.3789 47.2332,74.7079 47.3402,76.9952 47.4473,79.2419 47.5544,81.4488 47.6615,83.617 47.7686,85.7472 47.8757,87.8405 47.9828,89.8975 
 48.0899,91.9192 48.1969,93.9064 48.304,95.8598 48.4111,97.7802 48.5182,99.6684 48.6253,101.525 48.7324,103.351 48.8395,105.146 48.9466,106.913 49.0536,108.65 
 49.1607,110.359 49.2678,112.041 49.3749,113.695 49.482,115.324 49.5891,116.926 49.6962,118.503 49.8033,120.056 49.9103,121.584 50.0174,123.089 50.1245,124.57 
 50.2316,126.029 50.3387,127.466 50.4458,128.881 50.5529,130.275 50.66,131.647 50.767,133 50.8741,134.332 50.9812,135.645 51.0883,136.939 51.1954,138.214 
 51.3025,139.471 51.4096,140.71 51.5167,141.931 51.6237,143.134 51.7308,144.321 51.8379,145.491 51.945,146.645 52.0521,147.783 52.1592,148.906 52.2663,150.013 
 52.3734,151.105 52.4804,152.182 52.5875,153.245 52.6946,154.293 52.8017,155.328 52.9088,156.349 53.0159,157.357 53.123,158.351 53.2301,159.333 53.3371,160.302 
 53.4442,161.259 53.5513,162.203 53.6584,163.136 53.7655,164.057 53.8726,164.966 53.9797,165.864 54.0868,166.751 54.1938,167.627 54.3009,168.493 54.408,169.347 
 54.5151,170.192 54.6222,171.027 54.7293,171.851 54.8364,172.666 54.9435,173.471 55.0505,174.267 55.1576,175.053 55.2647,175.831 55.3718,176.599 55.4789,177.359 
 55.586,178.11 55.6931,178.852 55.8002,179.587 55.9073,180.313 56.0143,181.03 56.1214,181.74 56.2285,182.443 56.3356,183.137 56.4427,183.824 56.5498,184.504 
 56.6569,185.176 56.764,185.841 56.871,186.499 56.9781,187.15 57.0852,187.795 57.1923,188.432 57.2994,189.063 57.4065,189.687 57.5136,190.305 57.6207,190.917 
 57.7277,191.523 57.8348,192.122 57.9419,192.715 58.049,193.303 58.1561,193.884 58.2632,194.46 58.3703,195.031 58.4774,195.595 58.5844,196.154 58.6915,196.708 
 58.7986,197.257 58.9057,197.8 59.0128,198.338 59.1199,198.871 59.227,199.399 59.3341,199.922 59.4411,200.44 59.5482,200.953 59.6553,201.462 59.7624,201.966 
 59.8695,202.465 59.9766,202.96 60.0837,203.451 60.1908,203.937 60.2978,204.418 60.4049,204.896 60.512,205.369 60.6191,205.838 60.7262,206.303 60.8333,206.764 
 60.9404,207.221 61.0475,207.674 61.1545,208.123 61.2616,208.568 61.3687,209.01 61.4758,209.448 61.5829,209.882 61.69,210.312 61.7971,210.739 61.9042,211.162 
 62.0112,211.582 62.1183,211.999 62.2254,212.412 62.3325,212.822 62.4396,213.228 62.5467,213.631 62.6538,214.031 62.7609,214.428 62.8679,214.822 62.975,215.212 
 63.0821,215.6 63.1892,215.984 63.2963,216.366 63.4034,216.744 63.5105,217.12 63.6176,217.493 63.7246,217.863 63.8317,218.23 63.9388,218.594 64.0459,218.956 
 64.153,219.315 64.2601,219.671 64.3672,220.024 64.4743,220.375 64.5813,220.724 64.6884,221.07 64.7955,221.413 64.9026,221.754 65.0097,222.092 65.1168,222.428 
 65.2239,222.762 65.331,223.093 65.438,223.422 65.5451,223.749 65.6522,224.073 65.7593,224.395 65.8664,224.715 65.9735,225.032 66.0806,225.348 66.1877,225.661 
 66.2947,225.972 66.4018,226.281 66.5089,226.588 66.616,226.893 66.7231,227.195 66.8302,227.496 66.9373,227.795 67.0444,228.091 67.1514,228.386 67.2585,228.679 
 67.3656,228.97 67.4727,229.259 67.5798,229.546 67.6869,229.831 67.794,230.115 67.9011,230.396 68.0081,230.676 68.1152,230.954 68.2223,231.23 68.3294,231.505 
 68.4365,231.778 68.5436,232.049 68.6507,232.318 68.7578,232.586 68.8648,232.852 68.971

I specifically chose boundary conditions that mimicked our original model. Compare this to the output from our first multibox run, and you'll see it's very similar. The second box peaks around a concentration of 3, the third box around a concentration of 2, and all boxes eventually converge on a concentration of 1.